In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
df = pd.read_csv('data_collection/data/database.csv')

In [4]:
tokenized = df['annotation'].apply(lambda x: tokenizer(x, padding=True, truncation=True, return_tensors='pt'))

In [5]:
vector_list = []
with torch.inference_mode():
    for i in range(tokenized.shape[0]):
        model_output = model(**{k: v for k, v in tokenized[i].items()})
        
        embeddings = model_output.last_hidden_state[:, 0, :]
        vector = torch.nn.functional.normalize(embeddings)[0].cpu().numpy()
        vector_list.append(vector)
        
vec_df = pd.DataFrame(vector_list)
vec_df.to_csv('vec_df.csv')

In [6]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [7]:
def recommend(user_query, top_n, model, tokenizer, vec_df, df):

    test_vec = embed_bert_cls(user_query, model, tokenizer)
    ret=cosine_similarity(vec_df, pd.DataFrame(test_vec).transpose())
    ret_df = pd.DataFrame(ret)
    ret_index = ret_df.sort_values(by=0, ascending=False).head(top_n).index
    result = df.iloc[ret_index].copy() #.to_dict()
    
    return result, ret_index.to_list()

In [8]:
query = 'историческая проза войны сражения герои'
output, ind = recommend(query, 3, model, tokenizer, vec_df, df)

for i in ind:
    print("Автор:", output['author'][i],'\n', "Название:", output['title'][i],'\n', "Аннотация:", output['annotation'][i], '\n\n')

Автор: Шолохов М.А. 
 Название: Они сражались за Родину 
 Аннотация: Роман выдающегося мастера литературы посвящен героическому подвигу советского народа в Великой Отечественной войне. В нем рассказано о нескольких днях жестоких боев, когда под непрерывными бомбежками и танковыми атаками фашистов стояла насмерть горсточка храбрецов. Эти люди — ярких характеров, разные по профессии, возрасту, темпераменту и национальности — в коротких передышках между боями говорят о войне и ее уроках, с юмором и подначками вспоминают мирную жизнь… В книгу также вошли рассказы «Судьба человека», «Наука ненависти» и очерки военных лет. 


Автор: Тёрнер Д. Т. 
 Название: Как Жить и в Мире, и в Эфире 
 Аннотация: Художественная литература.Захватывающий роман. 


Автор: Толстой  Л.Н. 
 Название: Война и мир. Том 4 
 Аннотация: Роман "Война и мир" по глубине и охвату событий до сих пор стоит на первом месте во всей мировой литературе. Все жизненные перипетии героев, происходящие на фоне исторических событий,